In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
import torch
from torch.utils.data import TensorDataset, DataLoader

In [3]:
df = pd.read_csv('default_of_credit_card_clients.csv')
df.drop(columns=['ID'],inplace=True)
target = 'default_payment_next_month'
target_rename = 'default'
df.rename(columns={target:target_rename}, inplace=True)
target=target_rename

In [4]:
def data_standardize(df=None,target=None):
    df_res = df.copy()
    for col in df_res.columns[df_res.columns != target]:
        df_res[col] = (df_res[col] - df_res[col].mean()) / df_res[col].std()
    return df_res

df_std = data_standardize(df, target=target)

In [5]:
def data_split(df=None, label=None, validation=False, train_size=0.8, random_state=42, tensor=False):
    if validation == False and tensor == False:
        x_train, x_test, y_train, y_test = train_test_split(df.iloc[:,df.columns != label], df.iloc[:,df.columns == label], 
                                                            test_size=(1-train_size), random_state=random_state)
        return x_train, x_test, y_train, y_test
    elif validation == True and tensor == True:
        x_train, x_val_te, y_train, y_val_te = train_test_split(df.iloc[:,df.columns != label], df.iloc[:,df.columns == label], 
                                                            test_size=(1-train_size), random_state=random_state)
        x_val, x_test, y_val, y_test = train_test_split(x_val_te, y_val_te, 
                                                            test_size=0.5, random_state=random_state)
        X_train = torch.Tensor(x_train.values)
        X_val = torch.Tensor(x_val.values)
        X_test = torch.Tensor(x_test.values)
        Y_train = torch.Tensor(y_train.values)
        Y_val = torch.Tensor(y_val.values)
        Y_test = torch.Tensor(y_test.values)
        return X_train, X_val, X_test, Y_train, Y_val, Y_test

x_train_nn, x_val_nn, x_test_nn, y_train_nn, y_val_nn, y_test_nn = data_split(df=df_std, label=target
                                                                                  ,validation=True, train_size=0.8
                                                                                  ,tensor=True)

In [6]:
nn_batch_size = 64
train_loader = DataLoader(TensorDataset(x_train_nn, y_train_nn), batch_size=nn_batch_size, shuffle=True)
val_loader = DataLoader(TensorDataset(x_val_nn, y_val_nn), batch_size=nn_batch_size, shuffle=True)
test_loader = DataLoader(TensorDataset(x_test_nn, y_test_nn), batch_size=nn_batch_size, shuffle=True)

In [13]:
y_val_nn.size()

torch.Size([3000, 1])

In [7]:
from torch import nn, optim

class MyNetwork(nn.Module):
    def __init__(self, input_feature, dim1, dim2, drop, output_feature):
        super(MyNetwork, self).__init__()
        self.model1 = nn.Sequential(
            nn.Linear(input_feature, dim1, bias=True),
            nn.ReLU()
        )
        self.model2 = nn.Sequential(
            nn.Linear(dim1, dim2, bias=True),
            nn.Dropout(drop),
            nn.ReLU()
        )
        self.model3 = nn.Sequential(
            nn.Linear(dim2, output_feature, bias=True),
        )   
    
    def forward(self, input):
        x1 = self.model1(input)
        x2 = self.model2(x1)
        output = self.model3(x2)
        return output

In [8]:
from torch.utils.tensorboard import SummaryWriter

def training_nn(epochs=500, learning_rate=0.01, loss_func='CE', optimizer_para='Adam',
                input_feature=23, output_feature=2, dim1=36, dim2=108, dropout=0.8, val_size = None,
                train_loader=None, val_loader=None, test_loader=None, device=None):
    # Initialize model
    mynetwork = MyNetwork(input_feature=input_feature, dim1=dim1, dim2=dim2, drop=dropout, output_feature=output_feature)
    mynetwork.to(device)

    # Define loss function & optimizer
    if loss_func == 'CE':
        loss_fn = nn.CrossEntropyLoss()
        loss_fn.to(device)
    if optimizer_para == 'Adam':
        optimizer = optim.SGD(mynetwork.parameters(), lr=learning_rate)

    # Epoches, training & validation
    total_training_step, total_val_step = 0, 0
    writer = SummaryWriter("../logs_train")
    for i in range(epochs):
        if (i+1)%100 == 0:
            print(f"------------epoch: {i+1}------------")
        """
        Training
        """
        mynetwork.train()
        for data in train_loader:
            inputs, labels = data
            inputs = inputs.view(-1, input_feature)
            labels = labels.squeeze().long()
            inputs = inputs.to(device)
            labels = labels.to(device)
            outputs = mynetwork(inputs)
            loss = loss_fn(outputs, labels.long())
            
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            total_training_step += 1
            if total_training_step % 100 == 0:
                print(f"Traning: {total_training_step}; Loss: {loss.item()}")
                writer.add_scalar("train_loss", loss.item(), total_training_step)

        """
        Validation
        """
        mynetwork.eval()
        total_test_loss, total_acc = 0, 0
        with torch.no_grad():
            for data in val_loader:
                inputs, labels = data
                inputs = inputs.view(-1, input_feature)
                labels = labels.squeeze().long()
                inputs = inputs.to(device)
                labels = labels.to(device)
                outputs = mynetwork(inputs)
                loss = loss_fn(outputs, labels)
                total_test_loss += loss.item()
                acc = (outputs.argmax(1) == labels).sum()
                total_acc += acc
        print(f"total test loss: {total_test_loss}")
        print(f"total accuracy: {total_acc/val_size}")
        writer.add_scalar("test_loss", total_test_loss, total_val_step)
        writer.add_scalar("test_accuracy", total_acc/val_size, total_val_step)
        total_val_step += 1

        torch.save(mynetwork, f"mynetwork_{i}.pth")

    writer.close()

In [9]:
training_nn(epochs=100, learning_rate=0.01, loss_func='CE', optimizer_para='Adam',
                input_feature=23, output_feature=2, dim1=36, dim2=108, dropout=0.8, val_size = 3000,
                train_loader=train_loader, val_loader=val_loader, test_loader=test_loader, device='cpu')

Traning: 100; Loss: 0.49957379698753357
Traning: 200; Loss: 0.46580809354782104
Traning: 300; Loss: 0.5168883800506592
total test loss: 23.744441151618958
total accuracy: 0.7836666703224182
Traning: 400; Loss: 0.5825622081756592
Traning: 500; Loss: 0.5340575575828552
Traning: 600; Loss: 0.44454991817474365
Traning: 700; Loss: 0.399914413690567
total test loss: 22.917233496904373
total accuracy: 0.7889999747276306
Traning: 800; Loss: 0.5146798491477966
Traning: 900; Loss: 0.5637843012809753
Traning: 1000; Loss: 0.3446252942085266
Traning: 1100; Loss: 0.5287122130393982
total test loss: 22.39302009344101
total accuracy: 0.793666660785675
Traning: 1200; Loss: 0.4272773265838623
Traning: 1300; Loss: 0.4492266774177551
Traning: 1400; Loss: 0.42827507853507996
Traning: 1500; Loss: 0.4130016565322876
total test loss: 22.066223055124283
total accuracy: 0.8013333082199097
Traning: 1600; Loss: 0.5021511316299438
Traning: 1700; Loss: 0.47557103633880615
Traning: 1800; Loss: 0.5340029001235962
tot